# Objectifs

- **manipulation de textes** en vue de leur préparation pour une tâche d'apprentissage automatique : manipuler du texte "authentique" français produit par des internautes, pour en produire des représentations qui mettent en oeuvre une réduction des variables textuelles à différents degrés d'agressivité ;

- **apprentissage automatique** : entraîner et évaluer des modèles de classification automatique des textes préparés ;

- **maîtrise d'outils** : utilisation de librairies qui implémentent des fonctionnalités de manipulation de texte (`spacy`, `nltk`), de données tabulaires (`pandas`) ou d'apprentissage automatique (`scikit-learn`).

# Description de la tâche

__Tâche :__ classification (catégorisation) multi-classe (trois classes)

__Type de données :__ texte, tweets

__Langue :__ français

__Output :__ polarité du texte: positif, négatif, neutre

# (Installation et) importation des outils nécessaires

In [33]:
import os
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from pprint import pprint

In [40]:
# nltk : en général limité quant au traitement du français
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import SnowballStemmer # ou: from nltk.stem.snowball import FrenchStemmer

In [28]:
# En ligne de commande
# !pip install -U spacy
## !python -m spacy download fr
# !python -m spacy download fr_core_news_sm

In [4]:
# spacy : bonne couverture du français ; très efficace
import spacy
nlp = spacy.load('fr')

**ATTENTION : Les données de test sont destinées à une évaluation FINALE du modèle entraîné. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit de les examiner. La sélection itérative du meilleur modèle se fera en évaluant les différentes variantes du modèle (en faisant varier des hyper-paramètres) sur un jeu de données de validation.**

# (A) Récupération et mise en forme des données

Données du défi [DEFT2015](https://deft.limsi.fr/2015/corpus.fr.php?lang=fr) : tweets rédigés en français, portant sur la thématique des changements climatiques. Tweets annotés selon leur polarité, pour la tâche 1 du défi : "Classification des tweets selon leur polarité. Étant donné un tweet, cette tâche consiste à le classer, selon l’opinion/sentiment/émotion qu'il exprime, en positif, négatif, neutre ou mixte (si le tweet contient à la fois un sentiment positif et un sentiment négatif)."

__ATTENTION : Ces jeux de données ont été mis à notre disposition EXCLUSIVEMENT à des fins pédagogiques par les organisateurs du défi DEFT 2015. Leur redistribution est formellement interdite et tout travail écrit (rapport de stage, article, etc.) produit sur la base de ces données devra citer les sources indiquées sur le [site Web de DEFT 2015](https://deft.limsi.fr/2015/corpus.fr.php?lang=fr).__

Les étiquettes (la vérité terrain) et le texte des tweets sont stockés séparément. Nous les regrouperons à partir de l'identifiant du tweet.

## 1. Étiquettes

In [5]:
def make_label_file_path(parent_label_directory, train_or_test):
    return os.path.join(parent_label_directory,
                        '{}_References'.format(train_or_test.title()),
                        'T1.txt')

In [6]:
def map_label_to_numeric(label):
    return 1 if label == '+' else 0 if label == '=' else -1

In [7]:
def get_labels(parent_label_directory, train_or_test):
    label_file = make_label_file_path(parent_label_directory, train_or_test)
    labels = pd.read_table(label_file, header=None, names=['id', 'polarity'])
    labels['polarity'] = labels['polarity'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    return labels

In [8]:
train_labels = get_labels('data', 'train')
test_labels = get_labels('data', 'test')

In [9]:
train_labels.head()

,polarity
id,
487349133460918272,1
487354248959918080,-1
487360225654374401,0
487387097222098944,-1
487387321537269761,0


In [62]:
train_labels.shape

(7929, 1)

## 2. Textes

In [11]:
def make_data_dir_path(parent_data_directory, train_or_test):
    return os.path.join(parent_data_directory,
                        'deft2015_{}_twitter_raw'.format(train_or_test.upper()))

In [12]:
def get_tweets(parent_data_directory, train_or_test):
    if train_or_test == 'test':
        train_or_test += 's' # incohérence dans le nommage des répertoires
    data_dir = make_data_dir_path(parent_data_dir, train_or_test)
    tweets = dict()

    for file_name in sorted(os.listdir(data_dir)):
        if file_name.endswith(".txt"):
            with open(os.path.join(data_dir, file_name), 'r') as f:
                text = f.read()
            id_tweet = int(os.path.splitext(file_name)[0])
            tweets[id_tweet] = text
    
    return (pd.DataFrame.from_dict(tweets, orient='index')
                        .rename(columns={0: 'text'}))

In [13]:
parent_data_dir = 'data/twitter'
train_tweets = get_tweets(parent_data_dir, 'train')
test_tweets = get_tweets(parent_data_dir, 'test')

In [14]:
train_tweets.head()

,text
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n"
487387097222098944,Développement durable ma gueule\n
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...


__*QUESTION : Combien de tweets y a-t-il dans le jeu de données d'entraînement et dans celui de test ?*__

In [15]:
len(train_tweets), len(test_tweets)

(7511, 3285)

## 3. Jonction des textes et des étiquettes

Il y a plus d'étiquettes que de textes, car des tweets ont pu disparaître entre le moment où ils ont été collectés pour l'annotation de référence et le moment où ils ont été récupérés ultérieurement (voir [ici](https://deft.limsi.fr/2015/evaluation.fr.php?lang=fr)). Nous ferons une jointure interne pour ne retenir que les éléments communs aux deux tableaux.

Cependant, tous les tweets disponibles ont une étiquette :

In [16]:
len(set(train_tweets.index).intersection(set(train_labels.index))) == len(train_tweets)

True

In [17]:
def merge_tweets_and_labels(tweets_df, labels_df):
    return pd.merge(tweets_df, labels_df, how='inner',
                    left_index=True, right_index=True)

In [18]:
train_tweets = merge_tweets_and_labels(train_tweets, train_labels)
test_tweets = merge_tweets_and_labels(test_tweets, test_labels)

In [19]:
train_tweets.head()

,text,polarity
487349133460918272,#Question orale à @RoyalSegolene au sujet de l...,1
487354248959918080,@PhanEd @AznAlainT @_Baekholic alors j'ai une ...,-1
487360225654374401,"""On peut vendre du vent, regarde les éoliennes""\n",0
487387097222098944,Développement durable ma gueule\n,-1
487387321537269761,Quand l'cosystme numrique bordelais rencontre ...,0


## 4. Exploration des données

### 4.1. Distribution des classes

In [20]:
class_distribution = (pd.DataFrame.from_dict(Counter(train_tweets.polarity.values),
                                             orient='index')
                                  .rename(columns={0: 'num_examples'}))
class_distribution.index.name = 'class'

In [21]:
class_distribution

,num_examples
class,
1,2364
-1,1763
0,3384


In [22]:
class_distribution['perc_examples'] = np.around(class_distribution.num_examples /
                                                np.sum(class_distribution.num_examples), 2)

In [31]:
class_distribution

,num_examples,perc_examples
class,,
1,2364,0.31
-1,1763,0.23
0,3384,0.45


### 4.2. Exploration du texte

**ATTENTION :** Ne pas regarder les données de test !

In [36]:
train_tweets['text'].values[:15]

array([ "#Question orale à @RoyalSegolene au sujet de l'efficacité énergétique de @MicheleBonneton http://t.co/RrHxGn5URS @Dailymotion\n",
       "@PhanEd @AznAlainT @_Baekholic alors j'ai une blague mais dure a comprendre: On est ecologiques a nous 3 on se rebelle groupe Anti-Train !\n",
       '"On peut vendre du vent, regarde les éoliennes"\n',
       'Développement durable ma gueule\n',
       "Quand l'cosystme numrique bordelais rencontre la mission French Tech nationale http://t.co/oCqL9tb4SZ\n",
       'Madame Ségolène Royale, ministre de l\'écologie et du développement durable : Non, à la destruction des "nuisibles" http://t.co/jUYWddlIMe\n',
       'Le nouveau Monsieur " développement durable ": Jacques Tapin, l’ex-élu municipal niortais, vient d’être porté ... http://t.co/hrDGKOkyJd\n',
       'Le ciment s’offre une empreinte carbone réduite: Si le ciment est connu pour ses qualités écologiques, il est ... http://t.co/Gd028bTRe2\n',
       "J'ai mis à jour mon profil Viadeo :

In [55]:
s = nlp('Le nouveau Monsieur " développement durable ": Jacques Tapin, l’ex-élu municipal niortais, vient d’être porté ... http://t.co/hrDGKOkyJd\n')
[ent.label_ for ent in s.ents]

['PER', 'PER', 'PER']

# Représentation des textes

## Descripteurs

**Du texte au numérique:**

- les descripteurs (variables, features, traits) sont des unités textuelles (lemmes, racines, autres ; prises individuellement ou en séquences, etc.) (mais on peut leur ajouter des variables non-textuelles) ;

- les valeurs de ces variables sont numériques : binaires, numériques discrètes, numérique continues.

## Texte vs corpus

Traditionnellement : les descripteurs textuels sont calculés sur l'ensemble du corpus. Tous les textes sont représentés par le mêmes ensemble de descripteurs, ce qui fait que la représentation d'un texte est un grand vecteur épars de taille fixe (taille du vecteur = taille du vocabulaire du corpus).

Approches plus récentes : chaque mot est représenté par un vecteur dense de valeurs réelles. Le texte est représenté par une aggrégation sous une certaine forme des représentations de ses mots constituants.

## 1. Sélection de descripteurs : prétraitements textuels

Objectif : réduire le nombre de descripteurs : réduire à un seul descripteur ceux qui sont équivalents ou qui peuvent être regroupés dans une classe d'équivalence.

Lemmatisation, racinisation, normalisation/correction ortho, suppression des accents, minuscules?, suppression de la ponctuation (penser si on veut en garder certains ; **traiter la négation à part !**), mots vides

Faire une regex, e.g. pour remplacer les adresses mail, les hashtags, les nombres, etc. par des placeholders. Classes de caractères Unicode (comprend des caractères non-ASCII) : `\w`, `\d`. OU BIEN : faire une REN avec `spacy` et remplacer les entités par leur classe.

In [57]:
# Exemple
tw = nlp(train_tweets['text'].iloc[0])
tw

#Question orale à @RoyalSegolene au sujet de l'efficacité énergétique de @MicheleBonneton http://t.co/RrHxGn5URS @Dailymotion

**N.B.** Dans la pratique certains ces procédés sont souvent appliqués ensemble ou bien ils peuvent être pris en charge par l'implémentation de la méthode d'apprentissage même, qui les applique en boîte noire.

### 1.1. Pas de sélection : mots tels quels

In [ ]:
for token in tw:
    print(token)

### 1.2. Réduction par regroupement

Calculer chaque fois l'importance de la réduction (entre la taille du vocabulaire réduit / taille du vocabulaire initial).

#### 1.2.1. Lemmes

In [ ]:
for token in tw:
    print(token.lemma_)

#### 1.2.2. Racines

In [ ]:
tw = tweets['text'].iloc[0]
tw

In [ ]:
stemmer = SnowballStemmer('french')

In [ ]:
tokenizer = TweetTokenizer()
tokenizer.tokenize(tw)

In [ ]:
for token in tokenizer.tokenize(tw):
    print(stemmer.stem(token))

#### 1.2.3. Étiquettes morphosyntaxiques

In [ ]:
for token in nlp(tw):
    print(token.pos_)

#### 1.2.4. Classe d'appartenance des entités nommées

Limites : reconnaissance imparfaite. Faire des essais pour appréhender les limites de l'outil (erreurs systématiques, etc.). Dans notre cas, il vaut mieux éliminer les URL avant d'appliquer la reconnaissance d'entités nommées.

In [58]:
tw.ents

(Question orale, @MicheleBonneton http://t.co/RrHxGn5URS)

In [59]:
[ent.label_ for ent in tw.ents]

['MISC', 'LOC']

### 1.3. Réduction par filtrage : suppression de certains mots

#### 1.3.1. Filtrage des mots par fréquence d'utilisation en langue générale : "mots vides"

Critère vague. La notion de mot vide peut varier selon le contexte.

#### 1.3.2. Filtrage des mots par contenu expressif : mots qui n'ont pas une polarité claire

#### Moyennant un dictionnaire

Par exemple : dictionnaire [FEEL](http://advanse.lirmm.fr/feel.php) (voir [article](https://hal-lirmm.ccsd.cnrs.fr/lirmm-01348016/document)) : liste de 14128 **lemmes** annotés en termes de polarité (positif/négatif) et de six émotions (joy, fear, sadness, anger, surprise, disgust).

Lire le dictionnaire et n'en garder que la colonne "polarité".

#### En exploitant les fonctionnalités d'un outil de TAL

## 2. Calcul des valeurs des descripteurs

### 2.1. Binaire : présence/absence

### 2.2. Numérique discret : décomptes

### 2.3. Numérique continu : pondérations diverses

## Pistes pour l'approfondissement (travail personnel)

1. Combiner des descripteurs textuels (matrice mot-document) avec d'autres descripteurs.

2. Pousser la réduction de variables plus loin : correction d'orthographe.

Construire explicitement la matrice document-termes. Les "termes" (c.à.d. mots) sont des variables auxquelles on peut rajouter d'autres variables, de nature non-textuelle.

Extraire un descripteur du fichier `json` correspondant à chaque tweet.

# Classification des textes

## Approche sans apprentissage automatique

Calcul d'un score de polarité pour chaque texte, selon la formule : (donner référence, cf. livre Bing Liu)

Cela revient à un filtrage des mots.

## Approche par apprentissage automatique

Nous entraînerons des modèles appartenant à quelques familles d'algorithmes. D'autres familles restent à explorer. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

### Mise en place du schéma d'apprentissage

Schéma général : apprentissage (> évaluation > apprentissage > évaluation) > prédiction

Boîte à outils (algorithmes pré-implémentés) : `scikit-learn`. **Schéma de fonctionnement :**

* création de l'objet estimateur : appel du constructeur de la classe pertinente, avec d'éventuels paramètres si valeurs autres que défaut ;

* si besoin de réduire les données :
    - création d'un objet `scaler` (réducteur des données) ;
    - entraînement du `scaler` sur les données d'entraînement : méthode `fit` de l'objet réducteur ;
    - réduction des données d'entraînement : méthode `transform` du réducteur ;
    - réduction des données de validation : méthode `transform` du réducteur (attention : même réducteur que pour les données d'entraînement !) ;

* apprentissage de l'estimateur sur les données d'entraînement (éventuellement réduites) : méthode `fit` de l'estimateur ;

* évaluation sur les données d'entraînement, de validation et/ou (uniquement si c'est le modèle final !) de test : méthode `score` de l'estimateur.

* prédiction  : méthode `predict` de l'estimateur.

**Métriques d'évaluation :**

* taux de bonne classification (accuracy) ;

* précision (precision) ;

* recall (rappel) ;

* score F1 ;

* aire sous la courbe ROC (ROC AUC).

### 1. Weak baselines

- Choix aléatoire (probabilités égales pour les trois classes) : accuracy = 1 / 3.

- Prédiction constante de la classe majoritaire : accuracy = `{{np.max(class_distribution['perc_examples'])}}`.

### 2. Strong baseline : méthode du lexique polarisé

### 3. Strong baseline : classifieur Naïve Bayes

### 4. SVM